# Código do TCC de Paulo de Tarso, da EMAp FGV

In [99]:
import nltk
#Se for preciso, executar o comando abaixo
#nltk.download()
import unicodedata
import string
import os
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.svm import SVC

## Códigos auxiliares
### Código para remover símbolos das strings

In [38]:
def clean_string(s):
    s = s.replace("\n"," ")
    return ''.join(x for x in unicodedata.normalize('NFKD', s)
                   if x in string.ascii_letters + " ").lower()

## Parâmetros

In [39]:
#Parâmetros
inicio_atas = 104 #Utilizarei as atas a partir de 2005
final_atas = 206

total_atas = final_atas - inicio_atas + 1
atas = range(inicio_atas,final_atas+1)

path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/Atas/"

## Tokenização dos textos

#### Obter textos dos arquivos

In [71]:
texts = []

#Ir para a pasta com os arquivos
os.chdir(path)

#Obter o texto dos arquivos
for n in atas:
    file = open("COPOM_" + str(n) + ".txt")
    texts.append(clean_string(file.read()))
    file.close()
    
#Voltar uma pasta
os.chdir("..")

#### Vetorizar os textos obtidos

#### Stopwords

In [72]:
sw = [clean_string(word) for word in 
             open("stopwords.txt",encoding="utf-8").read().splitlines()[1:]]

#### Vectorização

In [113]:
vectorizer = CV(stop_words=sw,min_df=50)
arrays = vectorizer.fit_transform(texts).toarray()
#d_arrays = dict(zip(range(inicio_atas,final_atas+1),arrays))

## Obter os valores das variações na meta da Selic em cada reunião do Copom

In [176]:
file = open("Cortes.txt")
cortes = [int(line.split(";")[2]) for line in file.read().splitlines()[1:]]
cortes = [1 if n != 0 else 0 for n in cortes]
cortes.reverse()
file.close()

## Métodos de classificação

### SVM

In [177]:
classifier = SVC()
n = 1000
score = 0
for i in range(n):
    #Separação em dados para treino e dados para teste do modelo
    X_train,X_test,y_train,y_test = tts(arrays,cortes,train_size=0.9)
    
    classifier.fit(X_train,y_train)
    score += classifier.score(X_test,y_test)
score /= n
print(score)

0.691636363636
